## Assignment #6 - the final assignment

Here we're going to go through the full Viterbi algorithm for finding CpG islands in the genome. You'll need to have the 'chr21.fa.gz' file and the 'cpg_genome.bed' file in the same directory or change the corresponding paths below. 

Most of the assignment is pre-programmed, building up to our final application of the HMM here. The code you need to fill in is inside the code block with the <font color='red'>RED header</font> text.

#### First, a quick utility function to print a matrix

In [1]:
def pretty_print(seq_A, seq_B, dp_matrix,pad=6,round_to=3):
    '''
    a new pretty print that allows is more more flexible formatting options
    '''
    print("".join([letter_in_B.rjust(pad) for letter_in_B in seq_B]))

    for row_label, row in zip(seq_A, dp_matrix):
        print(row_label + "".join([str(round(mat_value,round_to)).rjust(pad) for mat_value in row]))


## Background distributions

- Lets use the A/C/G/T distribution in and out of CpG islands as our emissions

- Very simple, but as you'll see, it works suprizingly well (why?)


In [2]:
import gzip # use the gzip library in our project

chromosome_21_sequence = "" # this variable stores our chromsome 21 sequence

with gzip.open('chr21.fa.gz', 'rt') as chromosome_file: 
    header = chromosome_file.readline() 
    
    for line in chromosome_file:
        chromosome_21_sequence += line.strip()

# save ourselves a headache when we run into lower and uppercase letters
chromosome_21_sequence = chromosome_21_sequence.upper()

In [3]:
print(len(chromosome_21_sequence))

46709983


## Now lets see how dinucleotide proportions look across the chromosome:

In [4]:
import numpy as np

# a matrix of our emissions
emissions_global = np.zeros((5,5), np.int64)

# a function to convert a base to an array index
def base_to_index(base):
    if base == 'A':
        return 0
    elif base == 'C':
        return 1
    elif base == 'G':
        return 2
    elif base == 'T':
        return 3
    return 4 # catch all the other bases here

for index in range(0,len(chromosome_21_sequence) - 2):
    dinuc = chromosome_21_sequence[index:index + 2]
    index_1 = base_to_index(dinuc[0])
    index_2 = base_to_index(dinuc[1])
    emissions_global[index_1,index_2] += 1

## How dinucleotide proportions look across chromosome 21

In [5]:
pretty_print("ACGTN","ACGTN", emissions_global,10)

         A         C         G         T         N
A   3915117   2038442   2774777   3092310        18
C   2921760   2043408    462299   2757749        28
G   2420557   1705746   2062676   2037399         3
T   2563211   2397643   2926603   3968871         2
N        19         5        26         1   6621311


#### Normalize by total counts:

In [6]:
total_sum = emissions_global.sum() # axis=1
emissions_global_normalized = emissions_global / total_sum


## Do we see the effect of CpG -> TpG?

In [7]:
pretty_print("ACGTN","ACGTN", emissions_global_normalized)

     A     C     G     T     N
A 0.084 0.044 0.059 0.066   0.0
C 0.063 0.044  0.01 0.059   0.0
G 0.052 0.037 0.044 0.044   0.0
T 0.055 0.051 0.063 0.085   0.0
N   0.0   0.0   0.0   0.0 0.142


## Now load up known CpGs to get a background nucleotide frequency model for CpG regions


In [8]:
## What if we look only at CpG islands?
cpg_only_emissions = np.zeros((5,5), np.int64)
cpg_pairs = []

with open("chr21_cpg_locations_ucsc.bed") as cpg_file:
    for line in cpg_file:
        start_position = int(line.split("\t")[1])
        end_position =   int(line.split("\t")[2])
        cpg_pairs.append([start_position,end_position])
        for index in range(start_position,end_position - 1):
            dinuc = chromosome_21_sequence[index:index + 2]
            index_1 = base_to_index(dinuc[0])
            index_2 = base_to_index(dinuc[1])
            cpg_only_emissions[index_1,index_2] += 1
            

## Our emissions for CpGs

In [9]:
total_sum = cpg_only_emissions.sum()
cpg_only_emissions_normalized = cpg_only_emissions / total_sum
pretty_print("ACGTN","ACGTN", cpg_only_emissions_normalized)

     A     C     G     T     N
A 0.025 0.042 0.059 0.017   0.0
C 0.054 0.126 0.101 0.062   0.0
G 0.052 0.117 0.121 0.046   0.0
T 0.012 0.057 0.057 0.031   0.0
N   0.0   0.0   0.0   0.0  0.02


## Create just a per-base (not dinucleotide) emission probability



In [10]:
# lets simply our emissions -- (why? think about it a little...) 
per_base_cpg = cpg_only_emissions[0:4,0:4].sum(axis=1)/cpg_only_emissions[0:4,0:4].sum()
per_base_not = emissions_global[0:4,0:4].sum(axis=1)/emissions_global[0:4,0:4].sum()

print(per_base_cpg)
print(per_base_not)

[0.14557031 0.35065711 0.34365213 0.16012045]
[0.29486326 0.20417831 0.20520508 0.29575334]


# Transition probabilities

- Intuition: how often (in nucleotides) do we think we'll want to switch underlying states? 
- Relatively how often do the states occur? 10% CpG islands? 50%? 99%?

In [11]:
transition_props = np.array([[0.9999,0.0001],[0.00001,0.99999]])
pretty_print("CN", "CN", transition_props,pad=8,round_to=5)

       C       N
C  0.9999  0.0001
N   1e-05 0.99999


# Log transform the emission and transition values ahead of time



In [12]:
per_base_cpg_log = np.log10(per_base_cpg)
per_base_not_log = np.log10(per_base_not)
transition_log = np.log10(transition_props)


# Subset to just a 1 million basepair region of Chromosome 21 (9M-10M)

In [13]:
# lets keep it light and breezy, and do just 9M-10M of Chr21...
chr21_9m_to_10m = chromosome_21_sequence[9000000:10000000]

np.random.seed(0) # make sure the random.choice function always chooses the same sequence of random bases below

# clean out any non ACGT bases
chr21_9m_to_10m = [x if x in "ACGT" else np.random.choice(["A","C","G","T"]) for x in chr21_9m_to_10m]

# a 2 row X 1M row matrix - we'll 
hmm_model = np.ones((2,len(chr21_9m_to_10m)))
hmm_model_traceaback = np.ones((2,len(chr21_9m_to_10m)),np.int64)

print(hmm_model.shape)

(2, 1000000)


## setup the first column

- The first column is the transition from the start to each state


In [14]:
# some assumed probabilities
hmm_model[0,0] = np.log10([0.1]) + per_base_cpg_log[base_to_index(chr21_9m_to_10m[0])]
hmm_model[1,0] = np.log10([0.9]) + per_base_not_log[base_to_index(chr21_9m_to_10m[0])]

# we're coming from the non-CPG state
hmm_model_traceaback[0,0] = 1
hmm_model_traceaback[1,0] = 1

# <font color='red'>Fill in the rest of the matrix</font> - 8 points

#### Step 1: Fill in best_weight_and_previous_score_to_CpG and best_weight_and_previous_score_to_NOT_CpG:

Here we'll fill in correct values need in the update proceedure of our HMM. The <font color='blue'>best_weight_and_previous_score_to_CpG</font> variable we need to fill in represents the maximum value of two possible conditions:

- (1.1) the log transition probability from a __CpG__ state to this __CpG__ state, added (as we're in log space) to the score at that previous __CpG__ entry in our matrix
- (1.2.) the log transition probability from a __non-CpG__ state to this __CpG__ state, added (as we're in log space) to the score at that previous __non-CpG__ entry in our matrix

The <font color='blue'>best_weight_and_previous_score_to_NOT_CpG</font> is the same process but for __transitioning to the non-CpG state__ in our current column i. 

As a reminder the ```np.max([list_value1, list_value2, etc])``` function finds you the maximum value from a list of numbers. 


#### Step 2: Fill in the second two values, hmm_model_traceaback for 0,i and 1,i:

- (2.1.) The same best score calculation from 1.1 above. This does not include the emission probabilities we add later on
- (2.2) The same best score calculationfrom 1.2 above. This does not include the emission probabilities we add later on

Consider using the ```np.argmax([list_value1, list_value2, etc])``` function finds you the __index__ (which state we are transitioning from) of the maximum value from a list of numbers. 


In [15]:
for i in range(1,len(chr21_9m_to_10m)):
    cpg_emission_of_this_base     = per_base_cpg_log[base_to_index(chr21_9m_to_10m[i])]
    not_cpg_emission_of_this_base = per_base_not_log[base_to_index(chr21_9m_to_10m[i])]
    
    transition_cost_to_cpg     = np.max([hmm_model[0,i-1] + transition_log[0,0],
                                         hmm_model[1,i-1] + transition_log[0,1]])
    
    transition_cost_to_non_cpg = np.max([hmm_model[0,i-1] + transition_log[1,0],
                                         hmm_model[1,i-1] + transition_log[1,1]])
    
    hmm_model[0,i] = transition_cost_to_cpg     + cpg_emission_of_this_base
    hmm_model[1,i] = transition_cost_to_non_cpg + not_cpg_emission_of_this_base
    
    hmm_model_traceaback[0,i] = np.argmax([hmm_model[0,i-1] + transition_log[0,0],
                                           hmm_model[1,i-1] + transition_log[0,1]])
    
    hmm_model_traceaback[1,i] = np.argmax([hmm_model[0,i-1] + transition_log[1,0],
                                           hmm_model[1,i-1] + transition_log[1,1]])

## Output the tail end of the matrix to see our probabilties

In [16]:
# just some of the final cells you can compare against
# if you overwrite this cell you can look at the results on github
print(hmm_model[0,-5:])
print(hmm_model[1,-5:])


[-596143.28760461 -596144.08320125 -596144.51543764 -596145.31103428
 -596145.72356494]
[-596139.522482   -596140.05155668 -596140.73937291 -596141.26844759
 -596141.95844233]


# Traceback to test how we're doing


In [17]:
# find the maximal score in the final column
final_max = np.argmax([hmm_model[:,-1]])

# create a matrix to store the maximal path back through the HMM
state_path = np.zeros((len(chr21_9m_to_10m)),np.int64)

# We also want to know how many times we've transitioned between
# the two hidden states (we're curious..). Keep track of that here:
row_pointer = final_max
switch = 0

# how fill in the traceback matrix going from the end of the matrix
# to the begining (the final -1 step). We update which state (row) we got
# our best score from, and then move to the previous column using that
# row's trackback value.
for i in range(len(chr21_9m_to_10m)-1,0,-1):
    state_path[i] = row_pointer
    if hmm_model_traceaback[row_pointer,i] != row_pointer:
        switch += 1
    row_pointer = hmm_model_traceaback[row_pointer,i]

# print how many of each state (0 == CpG, 1 == not) we found
print(np.unique(state_path,return_counts=True))

# print how many times we switched between states
print("\nWe switched states: " + str(switch) + " times")

(array([0, 1]), array([ 75560, 924440]))

We switched states: 86 times


# Output a confusion matrix and accuracy for our model

In [18]:
confusion_matrix = np.zeros((2,2))

truth_array = np.ones((len(chr21_9m_to_10m)),np.int64)

for x in cpg_pairs:
    if x[0]  > 9000000 and x[1] < 10000000:
        truth_array[x[0] - 9000000:x[1] - 9000000] = 0

for i in range(0,len(chr21_9m_to_10m)):
    confusion_matrix[truth_array[i],state_path[i]] += 1

pretty_print(["True ","False"], ["CpG","not"], confusion_matrix,pad=10,round_to=5)

print("\naccuracy : " + str((confusion_matrix[0,0]+confusion_matrix[1,1])/confusion_matrix.sum()))

print("\npositive predictive value (PPV): " + str(confusion_matrix[0,0]/(confusion_matrix[0,0]++confusion_matrix[1,0])))

       CpG       not
True     4881.0      35.0
False   70679.0  924405.0

accuracy : 0.929286

positive predictive value (PPV): 0.06459767072525145


Well talk about this in the next class, but this is... decent. The accuracy is high, though with class imbalance this isn't the best measure. PPV is pretty low (you wouldn't want to 'diagnose' a patient with a CpG), but again this is a very rough cut of a model. You're done!

